# SAM2 動画内物体検出・追跡 チュートリアル

このノートブックでは、SAM2を使用した動画内物体検出・追跡の基本的な使用方法を学びます。

## 目次
1. [環境準備](#環境準備)
2. [動画フレーム分割](#動画フレーム分割)
3. [基本的な物体検出](#基本的な物体検出)
4. [完全な動画追跡](#完全な動画追跟)
5. [結果の分析](#結果の分析)

## 環境準備

必要なライブラリをインポートし、SAM2の準備を行います。

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch

# プロジェクトルートを追加
project_root = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
sys.path.insert(0, project_root)

from src.sam2_utils import (
    show_mask, show_points, get_frame_names, 
    load_sam2_predictor, display_frame_with_points
)
from src.sam2_video_tracker import SAM2VideoTracker

print(f"プロジェクトルート: {project_root}")
print(f"PyTorch バージョン: {torch.__version__}")
print(f"使用デバイス: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 動画フレーム分割

まず、動画をJPEGフレームに分割する方法を確認します。

In [ ]:
# 動画フレーム分割の例
# 実際の動画ファイルがある場合のコード例

# from scripts.video_to_frames import video_to_frames
# 
# input_video = "path/to/your/video.mp4"
# output_dir = "input/sample_frames"
# 
# if os.path.exists(input_video):
#     frame_count = video_to_frames(input_video, output_dir, quality=95)
#     print(f"フレーム分割完了: {frame_count}フレーム")
# else:
#     print("動画ファイルが見つかりません")

print("動画フレーム分割のコマンド例:")
print("python scripts/video_to_frames.py input.mp4 input/dog_images/")

## フレームの確認

分割されたフレームを確認します。

In [ ]:
# フレームディレクトリを探す
sample_dirs = [
    "input/dog_images",
    "input/sample_frames", 
    "input/frames"
]

video_dir = None
for dir_path in sample_dirs:
    full_path = os.path.join(project_root, dir_path)
    if os.path.exists(full_path):
        frame_names = get_frame_names(full_path)
        if frame_names:
            video_dir = full_path
            break

if video_dir:
    print(f"使用するフレームディレクトリ: {video_dir}")
    print(f"フレーム数: {len(frame_names)}")
    print(f"最初の5フレーム: {frame_names[:5]}")
    
    # 最初のフレームを表示
    first_frame_path = os.path.join(video_dir, frame_names[0])
    image = Image.open(first_frame_path)
    
    plt.figure(figsize=(10, 6))
    plt.title("最初のフレーム")
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    
    print(f"画像サイズ: {image.size}")
else:
    print("フレームディレクトリが見つかりません。")
    print("まず動画をフレーム分割してください:")
    print("python scripts/video_to_frames.py <動画ファイル> input/dog_images/")

## 基本的な物体検出

1フレームで物体検出を試してみます。

In [ ]:
if video_dir:
    # SAM2トラッカーを初期化
    tracker = SAM2VideoTracker(model_size="tiny", device="cpu")
    
    # 動画を初期化
    frame_names = tracker.initialize_video(video_dir)
    
    # 検出したい座標を指定（画像の中央付近）
    image = Image.open(os.path.join(video_dir, frame_names[0]))
    center_x, center_y = image.size[0] // 2, image.size[1] // 2
    
    points = [[center_x, center_y]]  # 中央の座標
    labels = [1]  # Positive（検出対象）
    
    print(f"検出座標: {points}")
    
    # 物体を追加
    frame_idx, obj_ids, mask_logits = tracker.add_object_points(
        frame_idx=0,
        obj_id=0,
        points=points,
        labels=labels
    )
    
    # 結果を表示
    plt.figure(figsize=(12, 8))
    plt.title("SAM2 物体検出結果")
    
    # 元画像を表示
    plt.imshow(image)
    
    # 指定座標を表示
    points_array = np.array(points, dtype=np.float32)
    labels_array = np.array(labels, dtype=np.int32)
    show_points(points_array, labels_array, plt.gca())
    
    # セグメンテーションマスクを表示
    mask = (mask_logits[0] > 0.0).cpu().numpy()
    show_mask(mask, plt.gca(), obj_id=obj_ids[0])
    
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    print("✅ 物体検出完了")
else:
    print("❌ フレームディレクトリが見つかりません")

## 完全な動画追跡

全フレームに対して物体追跡を実行します。

In [ ]:
if video_dir:
    print("動画全体での物体追跡を開始...")
    
    # 動画全体に追跡を伝播
    video_segments = tracker.propagate_in_video()
    
    print(f"追跡完了: {len(video_segments)}フレーム処理")
    
    # いくつかのフレームの結果を表示
    sample_frames = [0, len(frame_names)//4, len(frame_names)//2, 3*len(frame_names)//4, -1]
    
    fig, axes = plt.subplots(1, len(sample_frames), figsize=(20, 4))
    
    for i, frame_idx in enumerate(sample_frames):
        if frame_idx == -1:
            frame_idx = len(frame_names) - 1
            
        # フレーム画像を読み込み
        image_path = os.path.join(video_dir, frame_names[frame_idx])
        image = Image.open(image_path)
        
        axes[i].imshow(image)
        axes[i].set_title(f"Frame {frame_idx}")
        axes[i].axis('off')
        
        # 最初のフレームには座標点も表示
        if frame_idx == 0:
            show_points(points_array, labels_array, axes[i])
        
        # マスクを表示
        if frame_idx in video_segments:
            for obj_id, mask in video_segments[frame_idx].items():
                show_mask(mask, axes[i], obj_id=obj_id)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ 動画追跡完了")
else:
    print("❌ フレームディレクトリが見つかりません")

## 結果の分析

追跡結果を分析し、統計情報を確認します。

In [ ]:
if video_dir and 'video_segments' in locals():
    # 結果を分析
    analysis = tracker.analyze_results(video_segments, frame_names)
    
    print("=== 追跡結果分析 ===")
    print(f"総フレーム数: {analysis['total_frames']}")
    print(f"処理済みフレーム数: {analysis['processed_frames']}")
    print(f"検出されたオブジェクト数: {len(analysis['objects_detected'])}")
    
    for obj_id, count in analysis['objects_detected'].items():
        print(f"  オブジェクト {obj_id}: {count}フレームで検出")
    
    # マスクカバレッジの統計
    if analysis['mask_coverage']:
        all_coverages = []
        for frame_coverages in analysis['mask_coverage'].values():
            for coverage in frame_coverages.values():
                all_coverages.append(coverage)
        
        if all_coverages:
            print(f"\n=== マスクカバレッジ統計 ===")
            print(f"平均カバレッジ: {np.mean(all_coverages):.2f}%")
            print(f"最小カバレッジ: {np.min(all_coverages):.2f}%")
            print(f"最大カバレッジ: {np.max(all_coverages):.2f}%")
            print(f"標準偏差: {np.std(all_coverages):.2f}%")
            
            # カバレッジの推移をプロット
            frame_indices = []
            coverages = []
            
            for frame_idx in sorted(analysis['mask_coverage'].keys()):
                frame_coverages = analysis['mask_coverage'][frame_idx]
                if frame_coverages:
                    frame_indices.append(frame_idx)
                    coverages.append(list(frame_coverages.values())[0])  # 最初のオブジェクトのみ
            
            plt.figure(figsize=(12, 4))
            plt.plot(frame_indices, coverages, 'b-', linewidth=2)
            plt.title("フレーム別マスクカバレッジの推移")
            plt.xlabel("フレーム番号")
            plt.ylabel("カバレッジ (%)")
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
    
    print("\n✅ 分析完了")
else:
    print("❌ 分析するデータがありません")

## 結果の保存

追跡結果を画像ファイルとして保存します。

In [ ]:
if video_dir and 'video_segments' in locals():
    # 結果保存ディレクトリ
    output_dir = os.path.join(project_root, "result", "notebook_results")
    
    print(f"結果を保存中: {output_dir}")
    
    # 最初の10フレームのみ保存（デモ用）
    sample_count = min(10, len(frame_names))
    
    tracker.save_results(
        video_dir=video_dir,
        frame_names=frame_names[:sample_count],
        video_segments={k: v for k, v in video_segments.items() if k < sample_count},
        output_dir=output_dir,
        show_initial_points=points_array,
        show_initial_labels=labels_array
    )
    
    print(f"✅ {sample_count}フレームを保存完了")
    print(f"保存先: {output_dir}")
    
    # 保存されたファイルを確認
    if os.path.exists(output_dir):
        saved_files = os.listdir(output_dir)
        print(f"保存されたファイル数: {len(saved_files)}")
        if saved_files:
            print(f"例: {saved_files[:3]}")
else:
    print("❌ 保存するデータがありません")

## まとめ

このチュートリアルでは以下を学習しました：

1. **SAM2の基本的な使用方法**
2. **座標指定による物体検出**
3. **動画全体での物体追跡**
4. **結果の分析と可視化**
5. **結果の保存**

### 次のステップ

- 異なる座標での物体検出を試す
- 複数のオブジェクトを同時追跡する
- より高精度なモデル（large）を試す
- GPU環境での高速処理を試す

### 参考資料

- [SAM2公式リポジトリ](https://github.com/facebookresearch/sam2)
- [元記事: SAM2動画内物体検出・追跡](https://qiita.com/Neckoh/items/xxx)
- [プロジェクトREADME](../README.md)